In [1]:
import json

In [2]:
import base64
import algosdk
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import *

In [3]:
from dotenv import load_dotenv

In [4]:
import sys, os 
sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

# Accounts

In [5]:
mnemonic1 = "wish duck spin web fiber claw loyal false trap damp tortoise duty rural hungry detail stone excess clock mass phrase name fury advance able tattoo"
mnemonic2 = "donate biology lonely adjust degree window unaware clinic feel denial ugly gasp spring kiwi they person cabbage guess case donate resource thumb depth abstract disagree"
mnemonic3 = "ivory another airport govern muffin liquid sauce false walk humor truck build magic nephew pear prize inherit uniform hybrid label oven neck vendor able issue"

In [6]:
mnemonic.to_private_key(mnemonic1)

'5PfQo43fKqiQUFI6zw3LScRevYlH1nTaChE9asl5fUAGyOnO4zi021Kza06B03Sfh/v6AsNXbHR6mmSW5EU+yw=='

In [7]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    # accounts[counter]['pk'] = mnemonic.to_public_key(m)
    accounts[counter]['sk'] = mnemonic.to_private_key(m)

    accounts[counter]['pk'] =algosdk.account.address_from_private_key(accounts[counter]['sk'])


    counter += 1

# Connect to Client

In [8]:
# Specify the node address and token.

algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

In [9]:
print("Account 1 address: {}".format(accounts[1]['pk']))
print("Account 2 address: {}".format(accounts[2]['pk']))

Account 1 address: A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U
Account 2 address: IW6MG6R6WRHA422TFQI3A4PMVEPLFJBZL7CWCYVCKS5VL2D6VZGOFNJWPE


# Create Asset (Minting)

* Account 1 creates an asset called latinum and
* sets Account 2 as the manager, reserve, freeze, and clawback address.

In [10]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()

# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=1000,
    default_frozen=False,
    unit_name="LATINUM",
    asset_name="latinum",
    
    manager=accounts[1]['pk'],
    reserve=accounts[1]['pk'],
    freeze=accounts[1]['pk'], 
    clawback=accounts[1]['pk'],


    url="https://path/to/my/asset/details",
    decimals=0)

# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])


In [11]:
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)
# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

Signed transaction with txID: SZH7EAFKW33NZUWVRTTBGJNVZTLT6Y3WYUGN52YO22WJTJALS4DA
TXID:  SZH7EAFKW33NZUWVRTTBGJNVZTLT6Y3WYUGN52YO22WJTJALS4DA
Result confirmed in round: 4


In [12]:
print("Transaction information: {}".format(
    json.dumps(confirmed_txn, indent=4)))

Transaction information: {
    "asset-index": 1004,
    "confirmed-round": 4,
    "pool-error": "",
    "txn": {
        "sig": "cLLZODof7Yj/rILE0mpQMNat+FnEppvdmNb40uTKRFCO0iEjxjs5P+tXEWGQCAHjz8ojEdy3mDANa2dxC9T5Bw==",
        "txn": {
            "apar": {
                "an": "latinum",
                "au": "https://path/to/my/asset/details",
                "c": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
                "f": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
                "m": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
                "r": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
                "t": 1000,
                "un": "LATINUM"
            },
            "fee": 1000,
            "fv": 3,
            "gen": "sandnet-v1",
            "gh": "FgyrZ8W3zu8N0GqSOokoiiLxUVU7SnUM67vBZLKKv5I=",
            "lv": 1003,
            "snd": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDO

In [16]:
#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break


In [17]:
try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

Asset ID: 1004
{
    "clawback": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "creator": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "manager": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "name": "latinum",
    "name-b64": "bGF0aW51bQ==",
    "reserve": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "total": 1000,
    "unit-name": "LATINUM",
    "unit-name-b64": "TEFUSU5VTQ==",
    "url": "https://path/to/my/asset/details",
    "url-b64": "aHR0cHM6Ly9wYXRoL3RvL215L2Fzc2V0L2RldGFpbHM="
}
Asset ID: 1004
{
    "amount": 1000,
    "asset-id": 1004,
    "is-frozen": false
}


# Change the Manager

The current manager(Account 2) issues an asset configuration transaction that assigns Account 1 as the new manager.

In [ ]:
# Keep reserve, freeze, and clawback address same as before, i.e. account 2
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# asset_id = 328952;

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    manager=accounts[3]['pk'],
    reserve=accounts[1]['pk'],
    freeze=accounts[1]['pk'],
    clawback=accounts[1]['pk'])
# sign by the current manager - Account 2
stxn = txn.sign(accounts[1]['sk'])
# txid = algod_client.send_transaction(stxn)
# print(txid)

In [ ]:
# Wait for the transaction to be confirmed
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)

Signed transaction with txID: UGAG7ARQ3XPWAVVMERIQGGVNSI6R24PB6I6UJCEONR5GTMSYVJFA
TXID:  UGAG7ARQ3XPWAVVMERIQGGVNSI6R24PB6I6UJCEONR5GTMSYVJFA
Result confirmed in round: 7


In [ ]:
# Check asset info to view change in management. manager should now be account 1
print_created_asset(algod_client, accounts[3]['pk'], asset_id)

# Opt-in

In [18]:
# OPT-IN

# Check if asset_id is in account 3's asset holdings prior
# to opt-in
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

account_info = algod_client.account_info(accounts[2]['pk'])
holding = None
idx = 0
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    idx = idx + 1
    if (scrutinized_asset['asset-id'] == asset_id):
        holding = True
        break

if not holding:

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=accounts[2]['pk'],
        sp=params,
        receiver=accounts[2]["pk"],
        amt=0,
        index=asset_id)
    stxn = txn.sign(accounts[2]['sk'])
    # Send the transaction to the network and retrieve the txid.
    try:
        txid = algod_client.send_transaction(stxn)
        print("Signed transaction with txID: {}".format(txid))
        # Wait for the transaction to be confirmed
        confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
        print("TXID: ", txid)
        print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

    except Exception as err:
        print(err)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, accounts[3]['pk'], asset_id)


Signed transaction with txID: ZBMFY5GUH4HO3O4G24S6MZZIA6CJIMVBZPG32WO5FHUK3SWL2B3Q
TXID:  ZBMFY5GUH4HO3O4G24S6MZZIA6CJIMVBZPG32WO5FHUK3SWL2B3Q
Result confirmed in round: 5


# Asset Transfer

In [19]:
# transfer asset of 10 from account 1 to account 3
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[2]["pk"],
    amt=100,
    index=asset_id)
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)
# The balance should now be 10.
print_asset_holding(algod_client, accounts[2]['pk'], asset_id)


Signed transaction with txID: DCSNMCFJH7BHKUL3242CZHL44NSI6DOLGQD3KVVH4E4XHBRKJ3GQ
TXID:  DCSNMCFJH7BHKUL3242CZHL44NSI6DOLGQD3KVVH4E4XHBRKJ3GQ
Result confirmed in round: 6
Asset ID: 1004
{
    "amount": 100,
    "asset-id": 1004,
    "is-frozen": false
}


# Asset Freeze

In [20]:
params = algod_client.suggested_params()

# The freeze address (Account 2) freezes Account 3's latinum holdings.

txn = AssetFreezeTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    target=accounts[2]["pk"],
    new_freeze_state=True
    )
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
except Exception as err:
    print(err)
# The balance should now be 10 with frozen set to true.
print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

Signed transaction with txID: IUDPW7ORZUV4WM4Y66IQOZ32SW45ARWYGTC7YD2RLX7TIOZKKEQA
TXID:  IUDPW7ORZUV4WM4Y66IQOZ32SW45ARWYGTC7YD2RLX7TIOZKKEQA
Result confirmed in round: 7


# Revoke Asset

In [21]:
# The clawback address (Account 2) revokes 10 latinum from Account 3 and places it back with Account 1.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Must be signed by the account that is the Asset's clawback address
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[1]["pk"],
    amt=100,
    index=asset_id,
    revocation_target=accounts[2]['pk']
    )
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
except Exception as err:
    print(err)
# The balance of account 2 should now be 0.
# account_info = algod_client.account_info(accounts[2]['pk'])
print("Account 2")
print_asset_holding(algod_client, accounts[2]['pk'], asset_id)

# The balance of account 1 should increase by 200 to 1000.
print("Account 1")
print_asset_holding(algod_client, accounts[1]['pk'], asset_id)

Signed transaction with txID: CKF6HTJQLPW2CVT3GDLAKYUFVIZPREUVY5HOHM2LBH73CMSNZ3UQ
TXID:  CKF6HTJQLPW2CVT3GDLAKYUFVIZPREUVY5HOHM2LBH73CMSNZ3UQ
Result confirmed in round: 8
Account 2
Asset ID: 1004
{
    "amount": 0,
    "asset-id": 1004,
    "is-frozen": true
}
Account 1
Asset ID: 1004
{
    "amount": 1000,
    "asset-id": 1004,
    "is-frozen": false
}


# Destroy Asset

In [22]:
# With all assets back in the creator's account,
# the manager (Account 3) destroys the asset.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

# Asset destroy transaction
txn = AssetConfigTxn(
    sender=accounts[3]['pk'],
    sp=params,
    index=asset_id,
    strict_empty_address_check=False
    )

# Sign with secret key of creator
stxn = txn.sign(accounts[3]['sk'])
# Send the transaction to the network and retrieve the txid.
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4)
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
except Exception as err:
    print(err)

# Asset was deleted.
try:
    print("Account 2 must do a transaction for an amount of 0, " )
    print("with a close_assets_to to the creator account, to clear it from its accountholdings")
    print("For Account 1, nothing should print after this as the asset is destroyed on the creator account")

    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    # asset_info = algod_client.asset_info(asset_id)
except Exception as e:
    print(e)

TransactionPool.Remember: transaction RGVIU4P3LCKTYRXQWKN2GSB6BNDLRXZRZRG6KSBDIUONV3NFURMA: this transaction should be issued by the manager. It is issued by J3YO6ADM3FVYTJGHZI4SQHOT6424JCVNC3LNITLG3T5FOXBYSOPNRUYHQA, manager key A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U
Account 2 must do a transaction for an amount of 0, 
with a close_assets_to to the creator account, to clear it from its accountholdings
For Account 1, nothing should print after this as the asset is destroyed on the creator account
Asset ID: 1004
{
    "amount": 1000,
    "asset-id": 1004,
    "is-frozen": false
}
Asset ID: 1004
{
    "clawback": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "creator": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "manager": "A3EOTTXDHC2NWUVTNNHIDU3UT6D7X6QCYNLWY5D2TJSJNZCFH3FYXDOC6U",
    "name": "latinum"